# 1. Load data

In [2]:
import pandas as pd

gtd=pd.read_csv('gtd_clean.csv',encoding='latin1',low_memory=False)

gtd.head()

,Unnamed: 0,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,1,1.970000e+11,1970,0,0,NaN,0,NaN,130,Mexico,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
1,5,1.970010e+11,1970,1,1,NaN,0,NaN,217,United States,...,"The Cairo Chief of Police, William Petersen, r...","""Police Chief Quits,"" Washington Post, January...","""Cairo Police Chief Quits; Decries Local 'Mili...","Christopher Hewitt, ""Political Violence and Te...",Hewitt Project,-9,-9,0,-9,NaN
2,6,1.970010e+11,1970,1,2,NaN,0,NaN,218,Uruguay,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
3,10,1.970010e+11,1970,1,1,NaN,0,NaN,217,United States,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
4,11,1.970010e+11,1970,1,6,NaN,0,NaN,217,United States,...,NaN,Committee on Government Operations United Stat...,"Christopher Hewitt, ""Political Violence and Te...",NaN,Hewitt Project,-9,-9,0,-9,NaN


# 2. Define funcions for boost algorithms

In [48]:
from imblearn.ensemble import EasyEnsemble
from imblearn.ensemble import BalanceCascade
from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import RandomOverSampler

# call adaboost method
def ada_boost(features,itarget,os=False):
  
    model=AdaBoostClassifier(random_state=1,n_estimators=800,learning_rate=0.9,algorithm='SAMME',
                             base_estimator=DecisionTreeClassifier(max_depth=3))
    
    model.fit(features,itarget)
    return model

# vectorize the output classes to get numerical values
def transformData(target):
    le = preprocessing.LabelEncoder()
    le.fit(target)
    target=le.transform(target)
    return target

# 3. split data by region and pass it for training

In [49]:
from pandas import Series as s
import random
from sklearn.model_selection import train_test_split
random.seed(1001)
    
# iterate through each region and predict terrorist group
for i in range(12):
    #Find region
    regionTxt=s.unique(gtd['region_txt'])[i]
    region=s.unique(gtd['region'])[i]
    #get region data
    regData=gtd[gtd['region']==2]
    # select features and target
    itarget=transformData(regData['gname'])
    features=regData[['country', 'attacktype1', 'iyear','targtype1','success']]
    #print region info
    print("Region "+str(region)+": "+regionTxt+' '+str(regData.shape))
    print("No of terrorist groups identified in the region: "+str(s.unique(regData['gname']).shape[0]))

    # split data
    X_train, X_test, y_train, y_test = train_test_split(features, itarget, test_size=0.30)

    #call adaboost
    model=ada_boost(X_train,y_train,os=True)

    print("Prediction score: "+str(model.score(X_test, y_test)*100)+'%')

Region 2: Central America & Caribbean (6462, 138)
No of terrorist groups identified in the region: 89
Prediction score: 76.5342960289%
